In [ ]:
#!pip install numpy tqdm matplotlib scipy numba

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import time
from numba import njit

from tqdm            import tqdm
from scipy           import signal
from scipy.signal    import sosfiltfilt as sff

from propagation import received_power, quad_distribute_power, propagation_delay, \
                        map_rx_config, map_tx_config

from parameter_measurement import rtof_d_roberts_measure, dflipflop_vec, counter_simulation_vec

In [ ]:
weather     = 'clear' 
temperature = 298 # Kelvin
daynight    = 'day_directsun'


daynight_noise_factors = dict()
daynight_noise_factors['day_directsun']   = 1.000 # 5100 uA
daynight_noise_factors['day_indirectsun'] = 0.145 # 740 uA
daynight_noise_factors['night']           = 0.010 # very small

# minmax bounds to be safe if you set the noise factor yourself
bg_current = (np.minimum(1, np.maximum(0, daynight_noise_factors[daynight]))*5100)*1e-6;

weather_attenuation_factors = dict()
weather_attenuation_factors['clear'] = 0.0
weather_attenuation_factors['rain']  = -0.05
weather_attenuation_factors['fog']   = -0.2

atten = weather_attenuation_factors[weather];

**trajectory**

In [ ]:
y = np.linspace(1,50,50) # "from 1 m to 50 m and taking one measurement every 5 cm." 
x = np.zeros((y.shape)); 

In [ ]:
a = np.load('optics/qrx_planoconvex.npz');
f_QRX, pd_snst, pd_gain, pd_dim, rx_P_rx_factor, rx_I_bg_factor, rx_thermal_factor1, rx_thermal_factor2 = map_rx_config(a);

In [ ]:
a = np.load('optics/tx_lambertian_20deg_2W.npz')
tx_ha, tx_pwr, tx_norm, tx_lambertian_order = map_tx_config(a);

# Simulate propagation

**channel gain**

get received optical power (whole detector) in Watts

In [ ]:
pwr   = received_power(x, y, 0, pd_dim/1000, tx_pwr, tx_norm, tx_lambertian_order, atten)

In [ ]:
delay = propagation_delay(x, y, 0)

In [ ]:
f_simulation = 1e10;           # [Hz], simulation clock freq that should capture light delays amounting to 1cm
t_simulation = 1/f_simulation; # [s] , simulation clock period

simulation_start  = t_simulation; # [s], t_simulation rather than 0 avoids artifacts and sim ends at simulation_stop this way
simulation_stop   = 0.011; # [s], see note above about pi/32
simulation_length = int(simulation_stop/t_simulation)
print("Simulation stop time:",simulation_stop)
print("Simulation length   :",simulation_length)

s_simulation   = np.linspace(simulation_start, simulation_stop, simulation_length)
                             
f_adc_clock  = 1e8;             # [Hz], measurement clock freq
t_adc_clock  = 1/f_adc_clock;   # [s] , measurement clock period

simclock_subsample_rate = int(f_simulation/f_adc_clock)

s_adc_clock = np.zeros_like(s_simulation)
for i in range(0, int(simclock_subsample_rate/2)):
    s_adc_clock[ i::simclock_subsample_rate] = 1;

s_adc_clock_lead  = s_adc_clock[1:]
s_adc_clock_lag   = s_adc_clock[0:-1]
s_adc_clock_re    = np.concatenate((np.asarray([False]), np.logical_and((1-s_adc_clock_lag), s_adc_clock_lead)))
del s_adc_clock, s_adc_clock_lead, s_adc_clock_lag

simulation preliminaries - other

In [ ]:
f_e  = 1.0e6; # [Hz], emitted wave freq, left TX
r    = 1999;  # unitless, heterodyning factor
N    = 10;    # unitless, averaging factor for left TX

c = 299792458 # [m/s] speed of light

### this factor is precomputed since it's the same for all links
# /16 due to C_T^2 in the thermal_factor2, each cell gets 1/4 of the total cap
thermal_and_bg_curr = rx_I_bg_factor*bg_current + temperature*(rx_thermal_factor1 + rx_thermal_factor2/16)

add_noise  = 1

# sim

In [ ]:
sos_lo_het = signal.butter(8, 2*(f_e*1.20)*t_simulation, 'low', output='sos')
sos_hi_het = signal.butter(8, 2*(f_e*0.80)*t_simulation, 'high', output='sos')

In [ ]:
iterations = 100;
d_becha    = np.zeros((iterations, x.shape[0]))
d_roberts  = np.zeros((iterations, x.shape[0]))

s_h = np.sin(2*np.pi* f_e*(r/(r+1))*s_simulation - np.pi/32)>0;
s_e = np.sin(2*np.pi* f_e *s_simulation - np.pi/32)>0;
s_gate = np.sin(2*np.pi* f_e*(1/(N*(r+1))) *s_simulation - np.pi/32)>0;
s_gate = s_gate[s_adc_clock_re]
numsamples = len(s_simulation)

tx = np.sin(2*np.pi* f_e *s_simulation[s_adc_clock_re] - np.pi/32);

@njit(parallel=True, fastmath=True)
def gen_rx(s_simulation, delay, f_e, rx_noise_stdev, pd_gain, numsamples, add_noise, rx_peakAmps):
    rx  = ((rx_peakAmps*(np.sin(2*np.pi*f_e*(s_simulation - 2*delay) - np.pi/32)) / 2) + add_noise * rx_noise_stdev * np.random.randn(numsamples))*pd_gain;
    return  rx

@njit(parallel=True, fastmath=True)
def s_r_zc(rx):
    return rx>0

for j in tqdm(range(0, iterations)):
    for i in range(0, x.shape[0]):
        d = delay[i]*c;
        rx_peakAmps    = pwr[i]*pd_snst
        rx_noise_stdev = np.sqrt(rx_P_rx_factor*pwr[i] + thermal_and_bg_curr); 

        #rx = ((rx_peakAmps*(np.sin(2*np.pi*f_e*(s_simulation - 2*delay[i]) - np.pi/32)) / 2) + add_noise * rx_noise_stdev * np.random.randn(numsamples))*pd_gain;
        rx  = gen_rx(s_simulation, delay[i], f_e, rx_noise_stdev, pd_gain, numsamples, add_noise, rx_peakAmps)
        rx  = sff(sos_lo_het, sff(sos_hi_het, rx));
        s_r = s_r_zc(rx)
        
        s_eh      = dflipflop_vec(s_e, s_h)[s_adc_clock_re]
        s_rh      = dflipflop_vec(s_r, s_h)[s_adc_clock_re]
        s_phi     = np.logical_xor(s_eh, s_rh);
        s_phi_pp  = s_phi*s_gate # clock applied implicitly with adc_clock_re

        count = counter_simulation_vec(s_phi_pp, s_gate)
        f_i = f_e/(r+1);
        phase_shift_est = 2*np.pi*(np.asarray(count)*f_i/(N*f_adc_clock))
        d_becha[j,i] = c*(phase_shift_est/(2*np.pi*2*f_e))
        del s_r

        tx1 = np.fft.fft(tx);
        tx1[0:int(tx1.shape[0]/2)] = 0;
        tx1 = np.fft.ifft(tx1);
        rx = np.fft.fft(rx[s_adc_clock_re]);
        rx[0:int(rx.shape[0]/2)] = 0;
        rx = np.fft.ifft(rx);
        phase_shift_diff_est = np.mean(np.angle(tx1 * np.conjugate(rx)));
        d_roberts[j,i] = c*(phase_shift_diff_est/(2*np.pi*2*f_e))
        #d_roberts[j,i] = -rtof_d_roberts_measure(s_simulation[s_adc_clock_re], rx[s_adc_clock_re], f_e, c)
        del rx

In [ ]:
np.savez('ranging.npz', 
        delay=delay, 
        c=c, 
        d_roberts=d_roberts, 
        d_becha=d_becha,
        pwr=pwr,
        f_e = f_e,
        f_adc_clock=f_adc_clock,
        r=r,
        N=N,
        daynight=daynight,
        weather=weather,
        temperature=temperature)